Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (100000, 28, 28) (100000,)
Validation set (5000, 28, 28) (5000,)
Test set (5000, 28, 28) (5000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (100000, 784) (100000, 10)
Validation set (5000, 784) (5000, 10)
Test set (5000, 784) (5000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### Logistic regression with L2 regularization

Beta = 0 ==> Test accuracy = 85.0%  
Beta = 0.01 ==> Test accuracy = 86.9%  
Beta = 0.001 ==> Test accuracy = 87.8%  
Beta = 10e-4 ==> Test accuracy = 87.5%

In [106]:
batch_size = 128
l2_beta = 10e-4

graph = tf.Graph()
with graph.as_default():

  # Datasets.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(
            batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + \
    l2_beta * tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [107]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 24.974960
Minibatch accuracy: 7.8%
Validation accuracy: 11.9%
Minibatch loss at step 500: 2.745125
Minibatch accuracy: 80.5%
Validation accuracy: 76.5%
Minibatch loss at step 1000: 1.957279
Minibatch accuracy: 72.7%
Validation accuracy: 78.9%
Minibatch loss at step 1500: 1.233956
Minibatch accuracy: 82.8%
Validation accuracy: 80.4%
Minibatch loss at step 2000: 0.942549
Minibatch accuracy: 82.8%
Validation accuracy: 80.6%
Minibatch loss at step 2500: 0.658843
Minibatch accuracy: 84.4%
Validation accuracy: 81.8%
Minibatch loss at step 3000: 0.746357
Minibatch accuracy: 84.4%
Validation accuracy: 81.5%
Test accuracy: 87.5%


### 1-hidden layer neural network with L2 regularization
Beta = 0 ==> Test accuracy: 88.8%  
Beta = 0.01 ==> Test accuracy: 92.8%  
Beta = 0.001 ==> Test accuracy: 92.9%  
Beta = 10e-4 ==> Test accuracy: 88.9%

In [108]:
batch_size = 128
num_hiden_nodes = 1024
l2_beta_nn = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  w1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hiden_nodes]))
  b1 = tf.Variable(tf.zeros([num_hiden_nodes]))
  w2 = tf.Variable(
    tf.truncated_normal([num_hiden_nodes, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  h1 = tf.matmul(tf_train_dataset, w1) + b1
  logits = tf.matmul(tf.nn.relu(h1), w2) + b2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + \
    l2_beta_nn * (tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction_h1 = tf.matmul(tf_valid_dataset, w1) + b1
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(valid_prediction_h1), w2) + b2)
  test_prediction_h1 = tf.matmul(tf_test_dataset, w1) + b1
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(test_prediction_h1), w2) + b2)

In [109]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 659.259399
Minibatch accuracy: 6.2%
Validation accuracy: 30.3%
Minibatch loss at step 500: 196.838623
Minibatch accuracy: 80.5%
Validation accuracy: 80.4%
Minibatch loss at step 1000: 114.128181
Minibatch accuracy: 78.9%
Validation accuracy: 81.7%
Minibatch loss at step 1500: 69.354912
Minibatch accuracy: 88.3%
Validation accuracy: 82.5%
Minibatch loss at step 2000: 41.612591
Minibatch accuracy: 89.1%
Validation accuracy: 83.2%
Minibatch loss at step 2500: 25.112751
Minibatch accuracy: 90.6%
Validation accuracy: 86.5%
Minibatch loss at step 3000: 15.541229
Minibatch accuracy: 86.7%
Validation accuracy: 86.8%
Test accuracy: 93.0%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

Batch size = 4 -> Batch accuracy 99.2% but validation accuracy of 77.0% and test of 84.1%

In [49]:
batch_size = 128
num_hiden_nodes = 1024
l2_beta_nn = 0

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  w1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hiden_nodes]))
  b1 = tf.Variable(tf.zeros([num_hiden_nodes]))
  w2 = tf.Variable(
    tf.truncated_normal([num_hiden_nodes, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  h1 = tf.matmul(tf_train_dataset, w1) + b1
  logits = tf.matmul(tf.nn.relu(h1), w2) + b2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + \
    l2_beta_nn * (tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction_h1 = tf.matmul(tf_valid_dataset, w1) + b1
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(valid_prediction_h1), w2) + b2)
  test_prediction_h1 = tf.matmul(tf_test_dataset, w1) + b1
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(test_prediction_h1), w2) + b2)

In [50]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):

    offset = (step % 4) * batch_size
    
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 245.437195
Minibatch accuracy: 12.5%
Validation accuracy: 34.2%
Minibatch loss at step 500: 0.706743
Minibatch accuracy: 99.2%
Validation accuracy: 76.3%
Minibatch loss at step 1000: 0.076224
Minibatch accuracy: 99.2%
Validation accuracy: 75.1%
Minibatch loss at step 1500: 0.023377
Minibatch accuracy: 99.2%
Validation accuracy: 75.4%
Minibatch loss at step 2000: 0.024743
Minibatch accuracy: 99.2%
Validation accuracy: 76.4%
Minibatch loss at step 2500: 0.022512
Minibatch accuracy: 99.2%
Validation accuracy: 76.4%
Minibatch loss at step 3000: 0.021665
Minibatch accuracy: 99.2%
Validation accuracy: 76.4%
Test accuracy: 82.5%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

With keep_prob = 0.5, dropout increases test accuracy from 82.5% to 85.2%

In [48]:
batch_size = 128
num_hiden_nodes = 1024
l2_beta_nn = 0

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  w1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hiden_nodes]))
  b1 = tf.Variable(tf.zeros([num_hiden_nodes]))
  w2 = tf.Variable(
    tf.truncated_normal([num_hiden_nodes, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  h1 = tf.matmul(tf_train_dataset, w1) + b1

  keep_prob = tf.placeholder(tf.float32)
  h1_drop = tf.nn.dropout(tf.nn.relu(h1), keep_prob)

  logits = tf.matmul(h1_drop, w2) + b2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + \
    l2_beta_nn * (tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction_h1 = tf.matmul(tf_valid_dataset, w1) + b1
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(valid_prediction_h1), w2) + b2)
  test_prediction_h1 = tf.matmul(tf_test_dataset, w1) + b1
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(test_prediction_h1), w2) + b2)

In [49]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):

    offset = (step % 4) * batch_size
    
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 574.603516
Minibatch accuracy: 8.6%
Validation accuracy: 42.2%
Minibatch loss at step 500: 5.767303
Minibatch accuracy: 96.1%
Validation accuracy: 78.3%
Minibatch loss at step 1000: 0.494693
Minibatch accuracy: 99.2%
Validation accuracy: 78.4%
Minibatch loss at step 1500: 0.309281
Minibatch accuracy: 98.4%
Validation accuracy: 79.3%
Minibatch loss at step 2000: 0.127779
Minibatch accuracy: 99.2%
Validation accuracy: 78.7%
Minibatch loss at step 2500: 0.089793
Minibatch accuracy: 98.4%
Validation accuracy: 78.9%
Minibatch loss at step 3000: 0.128678
Minibatch accuracy: 97.7%
Validation accuracy: 79.2%
Test accuracy: 86.3%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [46]:
batch_size = 128
num_hidden_nodes1 = 2048
num_hidden_nodes2 = 1024
l2_beta_nn = 1e-4
keep_prob = 0.5

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  w1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes1], stddev=0.1))
  b1 = tf.Variable(tf.zeros([num_hidden_nodes1]))
  w2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes1, num_hidden_nodes2], stddev=0.01))
  b2 = tf.Variable(tf.zeros([num_hidden_nodes2]))
  w3 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes2, num_labels], stddev=0.01))
  b3 = tf.Variable(tf.zeros([num_labels]))
  global_step = tf.Variable(0)
  
  
  # Training computation.
  h1 = tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1)
  h1_drop = tf.nn.dropout(h1, keep_prob)
  h2 = tf.nn.relu(tf.matmul(h1_drop, w2) + b2);
  h2_drop = tf.nn.dropout(h2, keep_prob)
  logits = tf.matmul(h2_drop, w3) + b3
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + \
      l2_beta_nn * (tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2) + tf.nn.l2_loss(w3))
  
  # Optimizer.
  learning_rate = tf.train.exponential_decay(0.9, global_step, 500, 0.65, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction_h1 = tf.matmul(tf_valid_dataset, w1) + b1
  valid_prediction_h2 = tf.matmul(tf.nn.relu(valid_prediction_h1), w2) + b2
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(valid_prediction_h2), w3) + b3)
  test_prediction_h1 = tf.matmul(tf_test_dataset, w1) + b1
  test_prediction_h2 = tf.matmul(tf.nn.relu(test_prediction_h1), w2) + b2
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(test_prediction_h2), w3) + b3)

In [47]:
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.935120
Minibatch accuracy: 9.4%
Validation accuracy: 11.0%
Minibatch loss at step 1000: 1.292353
Minibatch accuracy: 81.2%
Validation accuracy: 85.6%
Minibatch loss at step 2000: 1.033857
Minibatch accuracy: 85.9%
Validation accuracy: 87.3%
Minibatch loss at step 3000: 1.012184
Minibatch accuracy: 88.3%
Validation accuracy: 87.9%
Minibatch loss at step 4000: 0.966844
Minibatch accuracy: 85.9%
Validation accuracy: 88.1%
Minibatch loss at step 5000: 0.961202
Minibatch accuracy: 89.8%
Validation accuracy: 88.1%
Minibatch loss at step 6000: 0.911623
Minibatch accuracy: 89.8%
Validation accuracy: 88.1%
Minibatch loss at step 7000: 0.917758
Minibatch accuracy: 87.5%
Validation accuracy: 88.2%
Minibatch loss at step 8000: 0.856824
Minibatch accuracy: 91.4%
Validation accuracy: 88.2%
Minibatch loss at step 9000: 1.023242
Minibatch accuracy: 84.4%
Validation accuracy: 88.2%
Minibatch loss at step 10000: 0.852239
Minibatch accuracy: 90.6%
Validation accura